# Rapport de Projet INF421 : Algorithmes de Planification de Chemin
Auteur : Élève X

Date : Novembre 2025

## 1. Introduction
Ce projet a pour objectif de concevoir, implémenter et évaluer des algorithmes de planification de chemin pour robots ou drones. Le problème fondamental, défini comme "trouver un mouvement sans collision entre une configuration initiale et finale", est ici instancié dans un environnement rectangulaire 2D peuplé d'obstacles rectangulaires. Nous explorons deux heuristiques distinctes : le Particle Swarm Optimization (PSO), une méthode bio-inspirée, et le Rapidly-exploring Random Tree (RRT), une approche basée sur les graphes. Enfin, nous étendons l'étude au cas multi-robots avec contraintes de sécurité.
## 2. Mise en place de l'environnement
### Question 1 : Parsing et Initialisation
Nous avons développé une classe `Environment` et une fonction de parsing. Le fichier d'entrée est lu séquentiellement pour extraire les dimensions $(x_{max}, y_{max})$, les positions de départ/arrivée des deux robots, le rayon de sécurité $R$, et la liste des obstacles $(x_o, y_o, l_x, l_y)$.

Choix d'implémentation : Nous stockons les obstacles sous forme de liste de tuples. Cette structure simple permet une itération rapide pour la détection de collisions, suffisante pour le nombre d'obstacles dans les scénarios de test.
### Question 2 : Affichage
La visualisation est assurée par la bibliothèque `matplotlib`. L'environnement est représenté par une figure où les obstacles sont des `patches.Rectangle` bleus. Les chemins sont superposés sous forme de lignes brisées. Cette méthode permet un débogage visuel immédiat de la validité topologique des chemins.

## 3. Approche PSO (Particle Swarm Optimization)
### Question 3 : Définition des Particules et Fitness
Contrairement à l'optimisation de fonctions classiques où une particule est un point, ici une particule représente un chemin complet.

- **Structure de données** : Une particule est un vecteur de dimension $2 \times N$ correspondant aux coordonnées de $N$ points de passage (waypoints) intermédiaires. Le chemin complet est formé en reliant $Start \rightarrow w_1 \rightarrow ... \rightarrow w_N \rightarrow Goal$.
- **Fonction de Cout [méthode .evaluate()] (Objectif)** : Elle est définie comme la somme des distances euclidiennes entre points consécutifs (longueur du chemin) plus une pénalité conséquente (10000) en cas de collision avec un obstacle. La minimisation de cette fonction favorise les chemins courts et valides.

### Question 4 : Détection de Collision
Segment par segment, la méthode `check_collision` vérifie que les extrémités du segment ne sont pas dans l'obstacle, puis fait appel à la fonction `lines_intersect`, pour déterminer de façon exacte si le segment coupe l'un des quatre bords de l'obstacle. Elles sont disponibles dans `Geometry`.

### Question 5 : Pseudo-code 

```Initialiser S particules avec des positions (waypoints) et vitesses aléatoires 
Tant que k < MaxIterations:\\
    Pour chaque particule i:
        Calculer Fitness(x_i)
        Si Fitness(x_i) < Fitness(p_best_i): Mettre à jour p_best_i
        Si Fitness(x_i) < Fitness(g_best): Mettre à jour g_best
    Pour chaque particule i:
        Mettre à jour v_i selon Eq. (1) 
        Mettre à jour x_i selon Eq. (2) 
Retourner g_best et g_best_score
```

### Question 6 : Complexité
La complexité par itération est $O(S \cdot N \cdot O_{bst})$, où $S$ est le nombre de particules, $N$ le nombre de waypoints, et $O_{bst}$ le nombre d'obstacles (à cause du coût de vérification de collision).

### Question 7 & 8 : Implémentation et Résultats
L'implémentation vectorisée avec `numpy` accélère les calculs.

Choix des hyper-paramètres : $c_1 = c_2 = 1.5$ et $w=0.5$ offrent un bon compromis entre exploration (éviter les minima locaux) et exploitation (raffiner le chemin).

Observation : Le PSO de base converge vite mais reste souvent coincé dans des optima locaux (chemins "tendus" mais traversant un obstacle, ou contournant trop largement) car la surface de fitness est très accidentée.

### Question 9, 10, 11 : Améliorations (Restart, Recuit Simulé, Apprentissage Dimensionnel)
- **Random Restart**: Réinitialiser les positions permet de trouver d'autres topologies de chemin (passer à gauche ou à droite d'un obstacle). Cela améliore grandement la robustesse sur les scénarios complexes.
- **Recuit Simulé**: Accepter temporairement des solutions dégradées permet de "sortir" d'un obstacle. $T^0$ a été fixé haut puis décroît ($\beta = 0.95$).
- **Apprentissage Dimensionnel**: Cette technique a permis d'affiner les coordonnées individuelles des waypoints, utile pour "corner" les obstacles de près sans collision.

```Initialiser S particules avec des positions (waypoints) et vitesses aléatoires 
Tant que k < MaxIterations:
    Pour chaque particule i:
        Calculer Fitness(x_i)
        Si Fitness(x_i) < Fitness(p_best_i): 
            Mettre à jour p_best_i
            p_stagnation = 0
        Sinon: p_stagnation_i += 1

        Si Fitness(x_i) < Fitness(g_best): Mettre à jour g_best
        #Temperature Annealing
        Sinon : 
            delta = Fitness(g_best) - Fitness(x_i)
            Mettre à jour g_best avec une probabilité $exp(−delta/T)$
    
    #Dimensional Learning
    Pour chaque particule i qui stagne (p_stagnatio_i>5):
        Pour chaque point j de p_best_i:
            Remplacer temporairement p_best[j] par g_best[j]
            Si Fitness(temp_p_best_i) < Fitness(p_best_i) : Remplacer définitivement p_best_i[j]
    
    Pour chaque particule i:
        #Random Restart
        La réinitialiser (positions et vitesses) avec une probabilté 0.01
        Sinon : 
            Mettre à jour v_i selon Eq. (1) 
            Mettre à jour x_i selon Eq. (2)
    Réduire T = T * beta
Retourner g_best 
```

### Question 12 : Conclusion sur PSO
Le PSO fonctionne très bien pour les trajectoire simples : scénarios 0, 1 et 2.

Plus le chemin s'apparente à un labyrinthe, plus il devient difficile de trouver un chemin.

Améliorations proposées : 
- **Lisser la trajectoire** : Une fonction qui "lisse" la trajectoire. Basée sur l'inégalité triangulaire, cette méthode lisse la trajectoire en allant chercher, pour chaque point, le point le plus lointain qu'elle peut connecter sans rencontrer d'obstacles. ==> Trajectoires plus lisses et moins nerveuses.
- **Pénalité progressive** : La pénalité dépend de l'épaisseur du mur traversé. En effet, dans les scénarios trois et quatre, les particules ont une grande probabilité de traverser plusieurs obstacles. Les obstacles étant massifs, elles ont peu de chance d'en sortir en un tour, et même si elles se rapprochaient de la sortie, elles ne détecteraient pas de changement dans son score. Ainsi, une pénalité progressive permet de faire comprendre que l'on se rapporche de la sortie de l'obstacle.
- **Amélioration du recuit simulé** : garder en mémoire le vrai meilleur score, mais changer le chemin qui guide les autres

Avec ces améliorations, on arrive à résoudre le scénario 3, mais pas le scénario 4, qui est encore plus labyrinthique, même si on obtient tout de même un chemin plus provhe de la solution. Ainsi, le PSO est puissant pour l'optimisation continue mais s'adapte difficilement à des environnements labyrinthiques. En effet, l'aspect des pénalités l'empêchent de franchir des obstacles, ce qui rend le franchissements de gros obstacles difficiles. Nous avons donc besoin d'un autre algorithme pour résoudre ce type de problème de recherche de chemin.


Pour `num_particles=200`, `num_waypoints=20` et `max_iter=200` : 

Sans amélioration :
<div style="display: flex; gap:20px; padding-bottom: 20px;" >
<div style="text-align: center; width=40%;">
    <img src="assets/SC0-amelioless.png" width="40%" >
    <img src="assets/SC0-amelio.png" width="40%">
    <br>
    <em>Scénario 0 sans / avec amélioration</em>

</div>
<div style="text-align: center; width=40%;">
    <img src="assets/SC1-amelioless.png" width="40%">
    <img src="assets/SC1-amelio.png" width="40%">
    <br>
    <em>Scénario 1 sans / avec amélioration</em>

</div>
</div>
<div style="display: flex; gap:20px;padding-bottom: 20px;" >
<div style="text-align: center; width=40%;">
    <img src="assets/SC2-amelioless.png" width="40%">
    <img src="assets/SC2-amelio.png" width="40%">
    <br>
    <em>Scénario 2 sans / avec amélioration</em>

</div>
<div style="text-align: center;width=40%;">
    <img src="assets/SC3-amelioless.png" width="40%">
    <img src="assets/SC3-amelio.png" width="40%">
    <br>
    <em>Scénario 3 sans / avec amélioration</em>
 
</div>
</div>
<div style="display: flex; gap:20px;" >
<div style="text-align: center;">
    <img src="assets/SC4-amelioless.png" width="20%">
    <img src="assets/SC4-amelio.png" width="20%">
    <br>
    <em>Scénario 4 avec / sans amélioration</em>
</div>
</div>

## 4. Approche RRT (Rapidly-exploring Random Tree)
### Question 13 : Structure de données Arbre
L'arbre est constitué de nœuds. Chaque `Node` contient : 
1. Ses coordonnées $(x,y)$.
2. Une référence vers son parent (`parent`).
3. Le coût cumulé depuis la racine (`cost`).

### Question 14 : Reconstruction du chemin
On remonte simplement les pointeurs `parent` depuis le nœud final jusqu'à la racine (start), puis on inverse la liste.

### Question 15 : Pseudo-code RRT (avec Rewiring - RRT*)

```Arbre={Start}
Pour k=1 à Max_iterations:
    v_rand est choisi aléatoirement dans E
    v_nearest est le plus proche voisin de v_rand
    v_new=v_nearest + delta*(v_rand-v_nearest)/||v_rand-v_nearest||
    Si pas de collision entre v_new et v_rand:
        Ajouter parent.v_new=v_nearest et mettre à jour le coût
        Voisins={noeuds à distance delta_r de v_new}
        Pour voisin in Voisins:
            Si pas de collision entre v_new et voisin:
                Si le coût de v_new est meilleur en passant par voisin:
                    v_new.parent=voisin et mise à jour du coût
        Pour voisin in Voisins:
            Si voisin != v_new.parent et pas de collision entre v_new et voisin:
                Si le coût de voisin est meilleur en passant par v_new:
                    voisin.parent=v_new et mise à jour du coût
        Si Distance(v_new,goal)<delta_s et pas de collision entre v_new et goal:
            Ajouter goal à l'arbre avec goal.parent=v_new et mise à jour du coût
            Reconstituer le chemin et le renvoyer
```
    
### Question 16 : Complexité
Une itération coûte $O(N)$ pour la recherche du plus proche voisin (sans structure spatiale avancée) ou $N$ est le nombre d'itérations et $O(1)$ pour l'extension. La complexité totale est quadratique en nombre de nœuds : $O(N^2)$. En pratique comme on trouve rapidement l'objectif, cela n'est pas un souci.

### Question 17 & 18 : Résultats RRT
Le RRT explore l'espace beaucoup plus efficacement que le PSO pour trouver un chemin valide, même tortueux. L'efficacité viens de la méthode de mise à jour des coûts, inspirée de l'equation de Jacobi-Bellman.

<div style="display: flex; gap:20px; padding-bottom: 20px; justify-content: center;" >
<img src="photos/Comparatif_3.png" width="80%" >
</div>

### Question 19 : Optimisation de chemin (Smoothing)
Après avoir trouvé un chemin, on le parcourt linéairement, en considérant les points i,i+1,i+2. Des que le segment i,i+2 ne collisionne pas, on enlève i+1 au chemin. La complexité est linéaire en le nombre de points. On vérifie aussi le chemin de la toute fin. 

### Question 21 : Intelligent Sampling
Au lieu d'un échantillonnage uniforme, on biaise la distribution : avec une probabilité $p=0.3$, on choisit uniformément un obstacle, un segment et un couloir d'une largeur donnée à l'extérieur de ce segment. Cela permet de se rapprocher des bords, et d'avoir un biais envers les points les plus extrémaux. 

### Question 22 : Conclusion RRT
Le RRT (surtout sa version optimisée triangulairement et avec intelligent sampling) s'est avéré plus robuste que le PSO pour ce problème géométrique. Les temps de convergence sont drastiquement plus faibles, et dans les environnements compliqués, on a des succès (chemins valides) alors que le PSO ne renvoie pas de chemin valide, ou alors nécessite beaucoup de tuning des hyperparamètres. 

## 5. Le problème multi-robots
### Question 23 : Algorithme proposé
Pour le problème à 2 robots avec zone de sécurité $R$, nous proposons une **Découplée avec retard** :
1. Calculer le chemin optimal pour le Robot 1 et Robot 2 en utilisant RT, et sans considérer l'autre robot à chaque fois.
2. Tant que les robots de heurtent, retarder le Robot 2 de **t=safe_distance/v**.

Pour cela, nous avons créé une méthode **not_collide(path_1,path_2,t_2)** qui vérifie si les robots se rentrent dedans lorsqu'ils parcourent path_1 et path_2, le deuxième partant avec un retard de t_2. 
Lorsque l'environnement est grand, le fait de retarder le Robot 2 pour éviter leur collusion permet de s'éloigner de manière négligeable des situations ou les robots sont seuls. Cela nous donne la garantie d'obtenir une solution proche de l'optimalité. Nous pouvons visualiser une animation sur le fichier **test_joint.py**. 


### Question 24 : Résultats

Nous trouvons que sur le scénario_2 par exemple, il n'est pas nécessaire de prendre du retard dans la simulation effectuée. En effet, pour que deux robots de heurtent, il faut que jeurs chemins se croisent *au même moment*, ce qui est très peu probable. 

<div style="text-align: center;">
<div style="display: flex; gap:20px; padding-bottom: 20px; justify-content: center;" >
    <img src="photos/Joint_animation_1.png" width="40%" >
    <img src="photos/Joint_animation_2.png" width="40%">
</div>
    <em>Animation des deux robots - Scénario 2</em>
</div>


## 6. Question finale
### Question 25 : Utilisation de l'IA et Github
Pour ce projet, Github a servi à travailler en binôme, grâce aux fonctionalités push et pull. 

L'IA et les LLM ont permis de donner corps aux classes et de transformer le pseudo code en code. Une fois que nous avions listé et réfléchi à toutes les méthodes, variables, types dont nous avions besoin, nous nous sommes aidés de LLM pour coder le tout en python sans erreur. 

Cela nous a aussi été utile pour manier les bibliothèques telles que matplotlib. 

L'IA n'a pas remplacé la conception algorithmique mais a accéléré l'implémentation de la syntaxe et la rédaction du rapport.